# Stored procedures in Azure Cosmos DB

Let's have some fun with [stored procedures](https://docs.microsoft.com/azure/cosmos-db/stored-procedures-triggers-udfs)! They let you implement custom logic in Javascript functions that get directly executed on your Cosmos DB instance. Stored procedures are most useful when you need to run atomic transactions over multiple items.

We start by creating the resources we are going to need: a database and a container.

In [ ]:
import os
import time
import azure.cosmos

db_name = "spdbtest"
container_name = "spcltest"
partition_key_value = "1234"

# Create the database if it doesn't exist
try:
    cosmos_client.create_database(db_name)
    print('Database created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/partitionKey"))
print('Container created')

### Our first stored procedure...

Now that we have a container to work with, we can create a stored procedure named `createItem` that just stores any object passed as a parameter. It's nothing more that a Javascript function that uses [Cosmos DB's server-side API](https://azure.github.io/azure-cosmosdb-js-server/) to interact with the container it's running in:

In [ ]:
sp1_name = 'createItem'
sp1_definition = {
    'id': sp1_name,
    'serverScript': """function (itemToCreate) {

    var context = getContext();
    var container = context.getCollection();

    // create the new item
    container.createDocument(container.getSelfLink(),
        itemToCreate,
        function (err, itemCreated) {
            if (err) throw new Error('Error' + err.message);
            context.getResponse().setBody(itemCreated.id)
        });
}"""
}
container_client = database_client.get_container_client(container_name)
container_client.scripts.create_stored_procedure(sp1_definition)
print('Stored procedure "' + sp1_name + '" created')

### ... in action!

Can't wait to try it! Let's call it by passing some JSON object as a parameter:

In [ ]:
result = container_client.scripts.execute_stored_procedure(sp1_name, partition_key_value, {
    'name': 'Alice',
    'partitionKey': partition_key_value
})
print('New item with id "' + result + '" created')

Looks like our stored procedure has created an item based on the JSON object we passed. We can verify that by reading that new item and looking up the value of its `name` property:

In [ ]:
new_item = container_client.read_item(result, partition_key_value)
print('"name" value is: ' + new_item['name'])

### Making it more useful

Great! But not super interesting... We could have done that directly client-side without the need for a stored procedure. Let's spice things up with a second one that creates an item only if a pre-condition is met:

In [ ]:
sp2_name = 'createItemConditional'
sp2_definition = {
    'id': sp2_name,
    'serverScript': """function (nameValueToCheck, itemToCreate) {

    var context = getContext();
    var container = context.getCollection();
    
    var query = 
    {     
        'query' : 'SELECT * FROM c where c.name = @val',
        'parameters' : [{ 'name': '@val', 'value': nameValueToCheck }] 
    };
    // check for the pre-condition
    container.queryDocuments(container.getSelfLink(),
        query,
        function (err, items) {
            if (err) throw new Error('Error' + err.message);
            if (items.length > 0) {
                // if no item matches the pre-condition, create the new item
                container.createDocument(container.getSelfLink(),
                    itemToCreate,
                    function (err, itemCreated) {
                        if (err) throw new Error('Error' + err.message);
                        context.getResponse().setBody(itemCreated.id);
                    });            
            }
            else {
                context.getResponse().setBody('no item created');
            }
        });
}"""
}
container_client.scripts.create_stored_procedure(sp2_definition)
print('Stored procedure "' + sp2_name + '" created')

So this second stored procedure first searches for items where the value of the `name` property matches the first parameter. If any such document is found, it creates a new item from the JSON object passed as second parameter; if not, it does nothing.

Let's try it:

In [ ]:
result = container_client.scripts.execute_stored_procedure(sp2_name, partition_key_value, ['Bob', {
    'partitionKey': partition_key_value,
    'name': 'Bob'
}])
print(result)

Because our container currently doesn't contain any item where `name` equals `Bob`, the stored procedure didn't create any new item, as expected. But if we pass `Alice` as the first parameter, it will match the item we created just before, and a new item should be created:

In [ ]:
result = container_client.scripts.execute_stored_procedure(sp2_name, partition_key_value, ['Alice', {
    'partitionKey': partition_key_value,
    'name': 'Bob'
}])
print('New item with id "' + result + '" created')

It did! And just to be sure, let's fetch that new item to check the value of its `name` property:

In [ ]:
new_item = container_client.read_item(result, partition_key_value)
print('"name" value is: ' + new_item['name'])

### Stored procedure as atomic transactions

What's important to note here is that stored procedures are executed as **atomic transactions**, so there are only 2 situations possible:
- either the Javascript function succeeds and all the write operations it has performed get committed,
- or the Javascript function fails - because it has thrown an exception for example - and all the write operations it has performed get rolled back.

To illustrate that, let's rewrite our second procedure. This time, we invert the order of operations and start by writing the new item. Then, we check for the pre-condition and throw an exception if it's not met:

In [ ]:
sp3_name = 'createItemConditionalWithThrow'
sp3_definition = {
    'id': sp3_name,
    'serverScript': """function (nameValueToCheck, itemToCreate) {

    var context = getContext();
    var container = context.getCollection();

    // create the new item
    container.createDocument(container.getSelfLink(),
        itemToCreate,
        function (err, itemCreated) {
            if (err) throw new Error('Error' + err.message);
            context.getResponse().setBody(itemCreated.id);
            
            var query = 
            {     
                'query' : 'SELECT * FROM c where c.name = @val',
                'parameters' : [{ 'name': '@val', 'value': nameValueToCheck }] 
            };
            // check for the pre-condition
            container.queryDocuments(container.getSelfLink(),
                query,
                function (err, items) {
                    if (err) throw new Error('Error' + err.message);
                    if (items.length == 0) {
                        // if no item matches the pre-condition, throw an error
                        throw new Error('pre-condition not met');
                    }
                });
        });
}"""
}
container_client.scripts.create_stored_procedure(sp3_definition)
print('Stored procedure "' + sp3_name + '" created')

We try that new stored procedure, this time by encapsulating its call with a try-except block to catch an exception if thrown:

In [ ]:
try:
    result = container_client.scripts.execute_stored_procedure(sp3_name, partition_key_value, ['Carol', {
        'name': 'Carol',
        'partitionKey': partition_key_value
    }])
    print('New item with id "' + result + '" created')
except azure.cosmos.exceptions.CosmosHttpResponseError as e:
   print('Exception caught!')

And an exception got caught. We verify that the stored procedure was executed transactionally by fetching all items where `name` equals `Carol`. Although such item *was* written at the beginning of the function, this write operation must have been rolled back because of the exception.

In [ ]:
results = list(container_client.query_items(
    query='SELECT * FROM c WHERE c.name = "Carol"',
    enable_cross_partition_query=True))
print('Got ' + str(len(results)) + ' result(s)')

Indeed, we found no matching document.

Obviously, calling that last stored procedure with a matching pre-condition will succeed:

In [ ]:
try:
    result = container_client.scripts.execute_stored_procedure(sp3_name, partition_key_value, ['Bob', {
        'name': 'Carol',
        'partitionKey': partition_key_value
    }])
    print('New item with id "' + result + '" created')
except azure.cosmos.exceptions.CosmosHttpResponseError as e:
   print('Exception caught!')

new_item = container_client.read_item(result, partition_key_value)
print('"name" value is: ' + new_item['name'])

Learn more about [writing](https://docs.microsoft.com/azure/cosmos-db/how-to-write-stored-procedures-triggers-udfs) and [running](https://docs.microsoft.com/azure/cosmos-db/how-to-use-stored-procedures-triggers-udfs) stored procedures in Cosmos DB!

And before we close, don't forget to clean up the resources we've created:

In [ ]:
cosmos_client.delete_database(db_name)